# Implementacion del algoritmo CLR

In [70]:
# Dependencias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.linear_model import LassoCV
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [52]:
# Cargue de los datos | informacion
df = pd.read_excel("../Archivos Generados/DatasetFinal.xlsx")

In [53]:
# Se muestra los primeros 5 elementos
df.head(5)

,ID_LOTE,TIPO_SIEMBRA,SEM_TRATADAS,MATERIAL_GENETICO,CULT_ANT,DRENAJE,METODO_COSECHA,ALMACENAMIENTO_FINCA,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,...,Temp_Max_Avg_Mad,Temp_Min_Avg_Mad,Temp_Avg_Mad,Diurnal_Range_Avg_Mad,Sol_Ener_Accu_Mad,Temp_Max_34_Freq_Mad,Rain_Accu_Mad,Rain_10_Freq_Mad,Rhum_Avg_Mad,RDT_AJUSTADO
0,40,Mecanizado,NO,PIONEER 30F32,Algodon,SI,Manual,NO,5,63,...,32.05,23.60,27.83,8.45,13197.57,0.05,279.3,0.23,82.41,4767.44
1,43,Mecanizado,SI,DK 234,Maiz,SI,Manual,NO,5,64,...,32.37,23.49,27.93,8.89,12436.49,0.03,221.2,0.26,81.86,4651.16
2,44,Mecanizado,NO,PIONEER 30F32,Algodon,SI,Manual,NO,5,59,...,32.17,23.53,27.85,8.63,11267.17,0.03,226.0,0.27,82.61,5180.23
3,45,Mecanizado,NO,Otro,Algodon,SI,Manual,NO,5,64,...,32.19,23.54,27.86,8.65,11066.68,0.03,223.2,0.29,81.84,4897.67
4,46,Mecanizado,NO,Otro,Algodon,SI,Manual,NO,5,63,...,32.19,23.54,27.86,8.65,11066.68,0.03,223.2,0.29,81.84,5302.33


In [54]:
# Selección de las variables categoricas
cat_features = df.select_dtypes(include = ["object", "category"]).columns
cat_features

Index(['TIPO_SIEMBRA', 'SEM_TRATADAS', 'MATERIAL_GENETICO', 'CULT_ANT',
       'DRENAJE', 'METODO_COSECHA', 'ALMACENAMIENTO_FINCA',
       'TERRENO_CIRCUN_RASTA', 'POSICION_PERFIL_RASTA', 'PEDREG_PERFIL_ROCAS',
       'CAP_ENDURE_RASTA', 'MOTEADOS_RASTA', 'MOTEADOS_MAS70cm._RASTA',
       'ESTRUCTURA_RASTA', 'OBSERVA_EROSION_RASTA', 'OBSERVA_MOHO_RASTA',
       'OBSERVA_COSTRAS_DURAS_RASTA', 'SITIO_EXPUESTO_SOL_RASTA',
       'OBSERVA_COSTRAS_BLANCAS_RASTA', 'OBSERVA_COSTAS_NEGRAS_RASTA',
       'REGION_SECA_ARIDA_RASTA', 'OBSERVA_RAICES_VIVAS_RASTA',
       'OBSERVA_PLANTAS_PEQUENAS_RASTA', 'OBSERVA_HOJARASCA_MO_RASTA',
       'SUELO_NEGRO_BLANDO_RASTA', 'CUCHILLO_PRIMER_HTE_RASTA',
       'CERCA_RIOS_QUEBRADAS_RASTA', 'RECUBRIMIENTO_VEGETAL__SUELO_RASTA',
       'd.interno', 'drenaje_externo'],
      dtype='object')

In [55]:
# Codificación dummy de las variables categoricas
df_cat = pd.get_dummies(df[cat_features], drop_first = True, dummy_na = True)
df_cat.head()

,TIPO_SIEMBRA_Mecanizado,TIPO_SIEMBRA_nan,SEM_TRATADAS_SI,SEM_TRATADAS_nan,MATERIAL_GENETICO_ADV 9339 (Syngenta),MATERIAL_GENETICO_CORPOICA V 114,MATERIAL_GENETICO_Cerato (Syngenta),MATERIAL_GENETICO_DK 1040,MATERIAL_GENETICO_DK 1596,MATERIAL_GENETICO_DK 234,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [56]:
# Se eliminan las variables categoricas  del dataframe principal y se mantienen unicamente las variables numericas
df.drop(cat_features, axis = 1, inplace = True)

In [57]:
df.columns

Index(['ID_LOTE', 'DIAS_EN_EMERGER', 'DIAS_EN_EMERGER_A_FLORECER',
       'DIAS_EN_FLORECER_A_COSECHAR', 'POBLACION_20DIAS_AJT', 'ALTURA_LOT',
       'ContEnfQui_Emer_Flor', 'ContEnfQui_Flor_Cose', 'ContMalMec_Siem_Emer',
       'ContMalMec_Emer_Flor', 'ContMalMec_Flor_Cose', 'ContMalQui_Antes_Siem',
       'ContMalQui_Siem_Emer', 'ContMalQui_Emer_Flor', 'ContMalQui_Flor_Cose',
       'ContPlaQui_Antes_Siem', 'ContPlaQui_Siem_Emer', 'ContPlaQui_Emer_Flor',
       'ContPlaQui_Flor_Cose', 'TotN_Antes_Siem', 'TotN_Siem_Emer',
       'TotN_Emer_Flor', 'TotP_Antes_Siem', 'TotP_Siem_Emer', 'TotP_Emer_Flor',
       'TotK_Antes_Siem', 'TotK_Siem_Emer', 'TotK_Emer_Flor',
       'FerOrg_Emer_Flor', 'FerQui_Antes_Siem', 'FerQui_Siem_Emer',
       'FerQui_Emer_Flor', 'PENDIENTE_RASTA', 'NO_CAPAS_RASTA', 'PH_RASTA',
       'PROFUND_CAP_ENDURE_RASTA', 'ESPESOR_CAP_ENDURE_RASTA',
       'PROFUND_MOTEADOS_RASTA', 'PROFUND_RAICES_VIVAS_RASTA', 'prof_efectiva',
       'Porc_A', 'Porc_Ar', 'Porc_ArA', 'P

In [58]:
# Se construye un unico dataset | vista minable con valores numericos
df_final = pd.concat([df,df_cat], axis = 1)

In [59]:
df_final

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_ESPACIADO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_MUY BUENO,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_REGULAR,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_SIN COBERTURA,RECUBRIMIENTO_VEGETAL__SUELO_RASTA_nan,d.interno_EXCESIVO,d.interno_LENTO A MUY LENTO,d.interno_nan,drenaje_externo_NINGUNO,drenaje_externo_nan
0,40,5,63,68,60000,13,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,43,5,64,63,60000,15,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,44,5,59,66,60000,12,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,45,5,64,59,60000,12,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,46,5,63,60,60000,16,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,4378,7,47,84,70000,18,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
795,4379,5,47,81,62000,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,4380,6,49,79,61000,17,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
797,4382,7,48,95,65000,17,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [60]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Columns: 189 entries, ID_LOTE to drenaje_externo_nan
dtypes: float64(55), int64(31), uint8(103)
memory usage: 617.3 KB


# Modelado
## Regresion Lineal (Lasso)

In [61]:
# Division de los datos | variables dependientes
# X: Dependiente
# Y: Independiente
Y = df_final.RDT_AJUSTADO
X = df_final.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1)

In [62]:
Y

0      4767.44
1      4651.16
2      5180.23
3      4897.67
4      5302.33
        ...   
794    6418.60
795    5581.40
796    5106.98
797    5764.19
798    7341.86
Name: RDT_AJUSTADO, Length: 799, dtype: float64

In [63]:
# Se deviden los datos en conjunto de entranmiento y test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=40)
print(X_train.shape); print(X_test.shape)


(639, 187)
(160, 187)


In [64]:
# Implementación regresion lineal simple
lr = LinearRegression()
lr.fit(X_train, y_train)
# Obtencion de la metadata del modelo
lr.get_params(deep=True)

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False}

In [65]:
# Calculo del error 
pred_train_lr= lr.predict(X_train)
print("======================= Conjunto de entrenamiento ==========================")
print((mean_squared_error(y_train,pred_train_lr,squared=False)))
print(r2_score(y_train, pred_train_lr))
print("======================== Conjuneto de test  =================")
pred_test_lr= lr.predict(X_test)
print((mean_squared_error(y_test,pred_test_lr,squared=False))) 
print(r2_score(y_test, pred_test_lr))

======================= Conjunto de entrenamiento ==========================
648.3603502822376
0.780332724877175
======================== Conjuneto de test  =================
980.7553681822333
0.504322154691877


# Implementacion del regresor lineal lasso

-  Lasso internamente realiza selecccion de atributos dejando unicamente los atributos mas importantes
- Lasso es particularmente adecuado para situaciones en las que la cantidad de características es mayor que la cantidad de observaciones
- Para el caso del enfoque CLR es conveninte trabajar con algoritmos que no realizan seleccion de atributos
- Regresion lineal lasso es un tipo de regresion lineal regularizada.
- Los coeficientes del modelo se encuentran mediante un proceso de optimización que busca minimizar el error de suma cuadrada entre las predicciones ( yhat ) y los valores objetivo esperados ( y ). loss = sum i=0 to n (y_i – yhat_i)^2
- Una penalización L1 minimiza el tamaño de todos los coeficientes y permite que cualquier coeficiente alcance el valor cero, eliminando efectivamente las características de entrada del modelo
- Se utiliza un hiperparámetro llamado “ lambda ” que controla la ponderación de la penalización a la función de pérdida. Un valor predeterminado de 1,0 dará ponderación total a la penalización; un valor de 0 excluye la penalización. [entre 0,1]


**Observación** teniendo en cuenta que el algoritmo lasso realiza selección de atribributos, se realizan las pruebas pertinentes

In [66]:

# Se define el modelo
model = Lasso(alpha=1.0)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print("Vector MAE , en c/d iteración de la validación cruzada: ", scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Vector MAE , en c/d iteración de la validación cruzada:  [768.15621601 631.30273921 658.1481118  555.67760894 607.37863181
 671.12695857 768.12318003 647.65525441 667.38864302 610.45549141
 605.81682776 626.04287112 642.37740421 613.4548015  619.78440439
 751.44305232 729.58221554 749.34338298 666.26715831 655.67321992
 721.87616213 629.02759196 669.7876704  737.3529658  719.08543267
 628.75246278 665.38456555 557.30811746 633.45934383 689.39689639]
Mean MAE: 663.221 (56.935)


In [94]:
# Teniendo en cuenta el MAE se ajusta un modelo bajo esta regresión
# Se define el modelo
modelLasso = Lasso(alpha=0.78)
# fit model
modelLasso.fit(X_train, y_train)
# make a prediction
yhat = modelLasso.predict(X_test)
# summarize prediction
print('Predicted:', yhat)


Predicted: [3864.40142212 4504.12802402 5891.78104213 6186.3064082  3603.32705091
 3234.68049845 4195.85899911 6326.0813635  3695.51329718 3994.4484464
 4693.01333368 6998.92201556 4750.77377086 2579.07561772 4448.92845453
 3628.33157008 3749.20482711 3536.11619878 2510.14480984 3689.96295039
 5301.9037827  4873.04261933 3844.62128871 2693.17110539 6637.88143723
 6162.95297403 4913.35227526 5525.56178316 4462.83943879 6108.33368758
 2026.25616093 4044.27546445 4400.01357822 1773.25286302 5802.57182953
 5832.18047603 4816.35330463 4505.52885445 4367.7241921  4618.1295678
 4646.79783898 4829.42335804 4282.96785365 5295.05635071 3027.08727354
 3646.39096111 5288.66453163 3917.21852331 5743.14046387 6337.06859261
 4642.75133129 4208.71111419 4873.41318673 4531.64608273 2799.19506066
 5909.90883165 5301.51374555 4678.22723521 4753.2033525  5143.56215975
 5991.63328828 4709.76030543 4232.90104613 5650.66262048 6673.19092086
 4361.70527284 3995.81409061 6090.99833446 4842.1606411  6532.829688

C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.213e+07, tolerance: 1.223e+05
  model = cd_fast.enet_coordinate_descent(


In [95]:
# Coeficiente de determinacion de la predicción
R_2 = modelLasso.score(X_train, y_train)
print("El ajuste del modelo (Training ) es : ", R_2)
# Se calcula el error deel modelo
# Calculo del error 
print("======================= Conjunto de test ==========================")
print("MSE: ", (mean_squared_error(y_test,yhat,squared=False)))
print("MAE", mean_absolute_error(y_test,yhat))
print("r**2: ", r2_score(y_test, yhat))

El ajuste del modelo (Training ) es :  0.768140875884345
======================= Conjunto de test ==========================
MSE:  923.2894310846524
MAE 679.9351130438965
r**2:  0.5607074391781751


In [80]:
# Caracteristicas del modelo ajustado con Regresion Lineal Lasso
# Coeficientes del modelo ajustado..
modelLasso.coef_
# intercepto de la funcion de desicion
modelLasso.intercept_
# Cantidad de caracteristicas seleccionadas (Variables independientes)
modelLasso.n_features_in_
# Nombre de los atributos | caracteristicas seleccionadas
modelLasso.feature_names_in_


array(['DIAS_EN_EMERGER', 'DIAS_EN_EMERGER_A_FLORECER',
       'DIAS_EN_FLORECER_A_COSECHAR', 'POBLACION_20DIAS_AJT',
       'ALTURA_LOT', 'ContEnfQui_Emer_Flor', 'ContEnfQui_Flor_Cose',
       'ContMalMec_Siem_Emer', 'ContMalMec_Emer_Flor',
       'ContMalMec_Flor_Cose', 'ContMalQui_Antes_Siem',
       'ContMalQui_Siem_Emer', 'ContMalQui_Emer_Flor',
       'ContMalQui_Flor_Cose', 'ContPlaQui_Antes_Siem',
       'ContPlaQui_Siem_Emer', 'ContPlaQui_Emer_Flor',
       'ContPlaQui_Flor_Cose', 'TotN_Antes_Siem', 'TotN_Siem_Emer',
       'TotN_Emer_Flor', 'TotP_Antes_Siem', 'TotP_Siem_Emer',
       'TotP_Emer_Flor', 'TotK_Antes_Siem', 'TotK_Siem_Emer',
       'TotK_Emer_Flor', 'FerOrg_Emer_Flor', 'FerQui_Antes_Siem',
       'FerQui_Siem_Emer', 'FerQui_Emer_Flor', 'PENDIENTE_RASTA',
       'NO_CAPAS_RASTA', 'PH_RASTA', 'PROFUND_CAP_ENDURE_RASTA',
       'ESPESOR_CAP_ENDURE_RASTA', 'PROFUND_MOTEADOS_RASTA',
       'PROFUND_RAICES_VIVAS_RASTA', 'prof_efectiva', 'Porc_A', 'Porc_Ar',
       'Por

## Ajuste de Hiperparametros (alpha) | Obtención del mejor modelo

In [89]:
# Dependencias adicionales
import sklearn
from numpy import arange
from sklearn.linear_model import LassoCV

# Se define el metodo de evaluación del modelo (validacion cruzada)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# Se define el modelo
modelCv = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
# fit model
modelCv.fit(X, Y)


C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10718441.186295807, tolerance: 135424.0452085728
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21341033.437585205, tolerance: 135797.19253151235
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\German\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3558142.2043759227, tolerance: 139925.88889083528
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\German\AppData\Roaming\

LassoCV(alphas=array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99]),
        cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=1), n_jobs=-1)

In [93]:
# Propiedades del modelo
# Coeficientes del modelo
modelCv.coef_
# Valor de alfa seleccionado
print("Valor de alfa: ", modelCv.alpha_)
print("Version de Sklearn: ", sklearn.__version__)
#  Nombre de las caracteristicas seleccionadas 
modelCv.feature_names_in_
# Numero de caracteristicas seleccionadas
print("Numero de caracteristicas Seleccionadas: ", modelCv.n_features_in_)


Valor de alfa:  0.78
Version de Sklearn:  1.2.2
Numero de caracteristicas Seleccionadas:  187


### Conclusiones
- El regresor lineal lasso realiza internamente seleccion de atributos mediante un parametro de configuracion denominado alfa - [tasa de regularización], para este caso especifico despues de realizar la codificacion de los datos a formato dummy y pasar lso datos por el regresor se observa que se conservan las 187 caracteristicas (despues de la dumificación).
- El mejor valor de MSE obteniendo despues de encontrar  [alfa = 0.78] con la validación cruzada es de 680 logrando un ajuste unicamnete del 56% lo cual indica que es un modelo extremadamente malo.
- Se va a realizar las pruebas con el regresor Ridge y compara los respectivos resultados.


# Implementación del regreson Lineal Ridge
- Ridge es una regresión lineal regularizada que incluye una penalización L2.
- Los coeficientes de cada modelos de regresion lineal se encuentran mediante un proceso de optimización cuyo objetivo es minimizar el error de las predicciones yhat
- La caracteristica principal del regresor ridge es que agrega una penalizacion L2 para minimizar la suma de los coeficientes al cuadrado.
- Una penalización L2 minimiza el tamaño de todos los coeficientes, aunque evita que se eliminen del modelo al permitir que su valor se convierta en cero.
- La penalización de ridgue s maneja con un parametro de configuracion denominado (lamda) un valor predeterminado de 1,0 ponderará completamente la penalización; un valor de 0 excluye la penalización

In [97]:
# Dependencias necesarias
from sklearn.linear_model import Ridge


# Se define el modelo
model = Ridge(alpha=1.0)
# Se define el metodo de evaluación
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# Se evalua el modelo mediante la metrica MAE
scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = absolute(scores)
print("Vector de MAE: ", scores)
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

Vector de MAE:  [762.15140952 629.65478415 652.23632599 551.82672659 606.89283597
 673.96531923 770.03288884 636.90922474 661.64687451 598.09537759
 599.04874548 613.61085775 643.86601428 611.53087344 623.33273837
 744.45346341 722.81721908 746.43795707 640.92093595 655.24512255
 712.2878307  627.91020937 668.29897561 737.10487304 715.70390307
 627.82688349 663.8666971  556.78827648 622.92348923 680.15185942]
Mean MAE: 658.585 (57.058)


**Nota**  De manera analoga al caso anterior, se utiliza validacion cruzada con k-folder = 10 y 3 repticiones par obtner un estimado de lso limites de la metrica MAE. Esta metrica es el objetivo apra disminuir cuando se realize la implmentación del algorimo CLR

<te>